# Unsupervised Learning Awareness - Time Series use case

This notebook is following the progression of the Unsupervised Learning Awareness for AI Decision-makers. It provides practical illustrations in Python to understand the notions we have seen in this course.

<div class="alert alert-warning">
Author: Fabrice JIMENEZ
    
Link to course materials: https://github.com/jfabrice/ml-awareness-unsupervised-learning
</div>

## Preliminary loading with Google Colab

If you are using this notebook with Google Colab, please execute first the following cells, to retrieve the GitHub repository content and set the working directory. Otherwise, ignore these 3 cells and move to the next section.

In [ ]:
!git clone https://github.com/jfabrice/ml-awareness-unsupervised-learning.git

In [ ]:
import os
os.chdir('ml-awareness-unsupervised-learning')

In [ ]:
!pip install -r requirements.txt

# 1- Imports and dataset presentation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
df = pd.read_csv('data/dataset.csv')
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
print("Shape of our dataset: "+str(df.shape))

In [ ]:
df.head()

# 2- Preprocessing, data quality control

### High level exploration

How many cycles do we have? How many points per cycle?

In [ ]:
df.groupby('cycle').count()

There are 15 cycles, with very different number of points per cycle.
Let's see what our parameter looks like within a cycle.

In [ ]:
df[df['cycle']==1]['p1'].plot(style='o', alpha=0.4)

In [ ]:
df[df['cycle']==3]['p1'].plot(style='o', alpha=0.4)

The values seem to be continuous (close in time = close in value) which is logical if they are physical parameters.

The parameter seems to behave quite differently in different cycles.

### Missing values or uneven time steps

<div class="alert alert-warning">
Are there missing values?
</div>

In [ ]:
df.isnull().sum()

<div class="alert alert-warning">
Are the time steps all equal?
</div>

In [ ]:
timescale = pd.Series(df.index)
timescale.diff().value_counts()

Time steps are not always equal to 1 second...

Let's check an example of gap of 7 seconds, to see what strategy we could use.

In [ ]:
df['p1']['2001-01-10 00:10:30':'2001-01-10 00:11:30'].plot(style='o')

We see that values still seem to be continuous. This kind of gap can be interpolated with mean value.

<b>Conclusion</b>: We can resample data with equal steps of 1 second within each cycle, with mean interpolation (oversampling).

In [ ]:
newdf = []

# For each cycle, create a small dataframe with interpolation
for c in df['cycle'].unique():
    tmp = df[df['cycle'] == c].copy()
    tmp = tmp.resample('1S').mean().interpolate(method='linear')
    
    newdf.append(tmp)
    
# Concatenate the results to have the complete dataframe
newdf = pd.concat(newdf, axis=0)
df = newdf.copy()
del newdf

Let's check we only have time steps of 1 second or 1 day (between cycles)

In [ ]:
timescale = pd.Series(df.index)
timescale.diff().value_counts()

In [ ]:
df['p1']['2001-01-10 00:10:30':'2001-01-10 00:11:30'].plot(style='o')

We now have clearly defined samples (windows) which contain time series regularly indexed.

### Signal regularity

When we look at the parameter values closely, we have a lot of constant steps, making the overall shape piecewise constant.

In [ ]:
df[df['cycle'] == 1]['p1'][150:400].plot(style='o', alpha=0.4)

Intuitively, looking at the global shapes of signals, this kind of behavior will only add noise or pollution when we want to study the dynamics of the signal, for instance the frequencies, or the derivative. Let's filter our signals by applying a rolling mean, with a suitable window size making our signals smooth!

In [ ]:
df[df['cycle'] == 1]['p1'][150:400].rolling('10S').mean().plot(style='r-o', alpha=0.2)

Let's apply it on all cycles and all parameters.

In [ ]:
newdf = []

# For each cycle, create a small dataframe with rolling mean
for c in df['cycle'].unique():
    tmp = df[df['cycle'] == c].copy()
    tmp = tmp.rolling('10S').mean()
    
    newdf.append(tmp)
    
# Concatenate the results to have the complete dataframe
newdf = pd.concat(newdf, axis=0)
df = newdf.copy()
del newdf

df["cycle"] = df["cycle"].astype(int)

We now have smooth continuous time series for 15 flights, without missing values and with constant step size!

# 3- Feature engineering

For the task we need to achieve, we need to detect abnormal cycles. Therefore we need to have a dataset where each cycle is described by a set of features.

<div class="alert alert-warning">
What features can now be used to summarize each cycle?
</div>

To describe a cycle we want to estimate:
   - The distribution of parameter values: info contained in summary statistics of p1, p2, p3, p4 (mean, std...)
   - The dynamics - evolution of parameter values: info contained in the derivatives of these parameters

In [ ]:
# Function to be applied to the dataframe of each cycle
def computeFeatures(tab):
    res = pd.Series()
    
    # For each time series we compute the following features
    for p in [c for c in tab.columns if c != 'cycle']:
        # Mean
        res[p+'_mean'] = tab[p].mean()
        res[p+'_diff_mean'] = tab[p].diff().mean()
        # 1st decile
        res[p+'_d1'] = tab[p].quantile(0.1)
        res[p+'_diff_d1'] = tab[p].diff().quantile(0.1)
        # 9th decile
        res[p+'_d9'] = tab[p].quantile(0.9)
        res[p+'_diff_d9'] = tab[p].diff().quantile(0.9)
        # Standard deviation
        res[p+'_std'] = tab[p].std()
        res[p+'_diff_std'] = tab[p].diff().std()
    
    return res

We apply the feature computation on each cycle.

In [ ]:
features = df.groupby('cycle').apply(computeFeatures)
print("Shape of the features dataset: "+str(features.shape))

In [ ]:
features.head()

Now let's go back to class!

# 4- Pattern visualisation

Let's compute a PCA to look at patterns & correlations between our features!

In [ ]:
from pca import pca

In [ ]:
model = pca(n_components=5)
resPCA = model.fit_transform((features - features.mean())/features.std(), verbose=0)['PC']

In [ ]:
resPCA.head()

In [ ]:
fig,ax = model.plot()

In [ ]:
fig,ax = model.biplot(n_feat=5, legend=False)

In [ ]:
fig,ax = model.biplot3d(n_feat=5, legend=False)

Now let's go back to class!

# 5- Grouping similar cycles

We can perform clustering on the 5 PCA components, let's use a hierarchical clustering to see how many groups we should have.

In [ ]:
from scipy.cluster import hierarchy

In [ ]:
hac = hierarchy.linkage(resPCA, method='ward', metric='euclidean')
dendrogram = hierarchy.dendrogram(hac, labels=features.index.map(int))

A choice of 5 clusters seems reasonable! Let's cut the dendrogram at 4 groups.

In [ ]:
features['cluster'] = hierarchy.cut_tree(hac,5).flatten()

Let's visualize the clusters in the PCA projection.

In [ ]:
PCx = 'PC1'
PCy = 'PC2'

###############################
colors = ['blue','red','orange','cyan', 'magenta']
fig, ax = plt.subplots()
xl = plt.xlabel(PCx)
yl = plt.ylabel(PCy)
for i in range(len(resPCA)):
    row = resPCA.iloc[i]
    ax.scatter(row[PCx], row[PCy], c=colors[int(features['cluster'].iloc[i])], alpha=0.7)
    ax.text(x=row[PCx]+0.1, y=row[PCy]+0.1, s=str(int(features.index[i])))

We suspect cycles 3, 4 and 8 to be different from the others, let's see!

Now let's go back to class!

# 6- Identifying abnormal cycles

Since we don't have any reference of normality, we can directly exclude the novelty detection approach. We are forced to adopt outlier detection, considering that anomalies are cycles which are different from the majority.

Let's compute an Isolation Forest anomaly score and visualize it on the PCA projection.

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
clf = IsolationForest(n_estimators=200).fit(resPCA)

In [ ]:
features['score'] = clf.decision_function(resPCA)

In [ ]:
fig, ax = plt.subplots()
sc = ax.scatter(x=resPCA['PC1'],y=resPCA['PC2'], c=-features['score'], cmap='Reds')
fig.colorbar(sc, label='anomaly score')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title("Isolation Forest Score")

for i in range(len(resPCA)):
    row = resPCA.iloc[i]
    ax.text(x=row[PCx]+0.1, y=row[PCy]+0.1, s=str(int(features.index[i])))

In [ ]:
(-features['score']).plot(style='bo-', alpha=0.7)

We have our usual suspects: cycles 3 and 8 with good confidence, and cycles 4 and 15 with more uncertainty.

# 7- Validation

To validate our model, we need either: 
- a validation or test set, containing the ground truth (the clusters we had to find, the real anomalies...) - <b>Strong validation strategy</b>
- a human interpretation of the results (an expert eye who is able to tell if the clusters make sense, or if the anomalies detected are real ones...) - <b>Weak validation</b>

In the frame of this use case, we don't have any validation dataset, so we would rely only on a human interpretation by an expert. We would give the expert the parameters or behaviors we suspect to be anomalies. According to their interpretation, we can go back in the different steps of this analysis to change some elements (different resampling, smoothing, choice of features, method for clustering...)

This is why it is very important not to get lost in your choices! To be able to play with these choices, to adjust according to the results validation.

Now let's go back to class to conclude!